In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm

In [1]:
import rotation_matrix as rm

In [3]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [4]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [4]:
import numpy as np

data = np.loadtxt('../pouring_simulation/output/CellFlask/Flask_15_200_2/TCP.txt', delimiter=',', skiprows=1)

In [5]:
data.shape[0]

140

In [6]:
#start_pos = [-0.32840330749723673,
# 0.6906975268273259,
# 0.25610656751907307,
# -1.2222948307378443,
# -1.1666822479428505,
# -1.218172452957232]

#rtde_c.moveL(start_pos, 0.5, 0.5)

In [8]:

# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254

In [9]:
data

array([[2.42233196e-08, 8.29891660e-05, 5.00000000e-04],
       [8.47816440e-08, 1.66002462e-04, 1.00000000e-03],
       [1.93786506e-07, 2.48991628e-04, 1.50000000e-03],
       [3.39125560e-07, 3.32005940e-04, 2.00000000e-03],
       [5.08690880e-07, 4.14992820e-04, 2.50000000e-03],
       [7.38812340e-07, 4.98007640e-04, 3.00000000e-03],
       [1.01737922e-06, 5.80997060e-04, 3.50000000e-03],
       [1.32017262e-06, 6.64009340e-04, 4.00000000e-03],
       [1.68352216e-06, 7.46998760e-04, 4.50000000e-03],
       [2.08320640e-06, 8.29988180e-04, 5.00000000e-03],
       [2.51922534e-06, 9.13000460e-04, 5.50000000e-03],
       [2.99158660e-06, 9.95989880e-04, 6.00000000e-03],
       [3.50027240e-06, 1.07900470e-03, 6.50000000e-03],
       [4.06951180e-06, 1.16199412e-03, 7.00000000e-03],
       [4.67509860e-06, 1.24498100e-03, 7.50000000e-03],
       [5.31703280e-06, 1.32799582e-03, 8.00000000e-03],
       [5.99526360e-06, 1.41098524e-03, 8.50000000e-03],
       [6.72198300e-06, 1.49397

In [10]:
# start_pos = rtde_r.getActualTCPPose()

start_pos = [-0.34957822507297887,
 0.6821259171088855,
 0.22494216457758412,
 -1.2155551165943093,
 -1.176864822380718,
 -1.2292597140201593]

In [11]:
# create list of positions
positions = []
for i in range(data.shape[0]):
    positions.append([data[i,1], -data[i,0], 0.0, 0.0, 0.0, -data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

In [12]:
positions_converted = []
for i in range(data.shape[0]):
    positions_converted.append(rm.PoseTrans(start_pos, positions[i])) # transform from base to tool coordinates(positions[i]))

In [13]:
positions_converted

[[-0.3495775430212898,
  0.6821242425742096,
  0.22502513404786842,
  -1.2153281877133388,
  -1.1772338893846417,
  -1.2296307198508059],
 [-0.3495768970880237,
  0.6821225683946958,
  0.22510812796856908,
  -1.21510117590155,
  -1.1776029413176967,
  -1.2300017083342654],
 [-0.34957629979168087,
  0.6821208958251841,
  0.22519109820000024,
  -1.2148740811505216,
  -1.1779719781722027,
  -1.2303726794624938],
 [-0.3495757386043019,
  0.6821192235903084,
  0.2252740938975967,
  -1.214646903451837,
  -1.1783409999404737,
  -1.2307436332274537],
 [-0.3495752018691595,
  0.6821175524706786,
  0.2253570623872726,
  -1.2144196427970748,
  -1.1787100066148208,
  -1.2311145696211019],
 [-0.3495747254336669,
  0.6821158821907335,
  0.22544005935390177,
  -1.2141922991778096,
  -1.1790789981875531,
  -1.2314854886353936],
 [-0.3495742976447885,
  0.6821142135463989,
  0.2255230313615562,
  -1.213964872585615,
  -1.179447974650977,
  -1.23185639026228],
 [-0.3495738938803976,
  0.6821125450021964

In [17]:
# get first 5 x and z positions 
positions = positions_converted[15:30]

# only get x, y, z positions
positions = [x[0:3] for x in positions]


#positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions)-1):
    distances.append(np.linalg.norm(np.subtract(positions[i+1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i]*60)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

[8.299219137335695e-05, 8.299260179693007e-05, 8.301578659636623e-05, 8.299338726929587e-05, 8.299387527320199e-05, 8.299412961581579e-05, 8.301751832108308e-05, 8.299521847825158e-05, 8.299565608266385e-05, 8.29961089578174e-05, 8.299673871629219e-05, 8.299722866407036e-05, 8.30207652464592e-05, 8.299843534002689e-05]
[0.004979531482401417, 0.0049795561078158045, 0.004980947195781973, 0.004979603236157752, 0.0049796325163921195, 0.004979647776948947, 0.004981051099264985, 0.004979713108695095, 0.004979739364959831, 0.004979766537469043, 0.004979804322977531, 0.004979833719844222, 0.004981245914787553, 0.004979906120401613]
Speed in m/s:  0.004979998464564135


In [16]:
positions

[[-0.3495775430212898, 0.6821242425742096, 0.22502513404786842],
 [-0.3495768970880237, 0.6821225683946958, 0.22510812796856908],
 [-0.34957629979168087, 0.6821208958251841, 0.22519109820000024],
 [-0.3495757386043019, 0.6821192235903084, 0.2252740938975967],
 [-0.3495752018691595, 0.6821175524706786, 0.2253570623872726],
 [-0.3495747254336669, 0.6821158821907335, 0.22544005935390177],
 [-0.3495742976447885, 0.6821142135463989, 0.2255230313615562],
 [-0.3495738938803976, 0.6821125450021964, 0.22560602644122482],
 [-0.3495735508479827, 0.6821108783230521, 0.22568899921003613],
 [-0.3495732441390409, 0.6821092124861152, 0.22577197230505106],
 [-0.3495729735590255, 0.6821075470299678, 0.2258549685807843],
 [-0.3495727395046477, 0.6821058828776227, 0.2259379423282748],
 [-0.349572541547423, 0.682104219054563, 0.22602094179578297],
 [-0.34957240434125136, 0.682102557147771, 0.22610391641302047],
 [-0.3495723034928653, 0.6821008961347498, 0.22618688881718507]]

In [14]:
velocity = 0.5
acceleration = 0.5
blend_1 = 0.0
blend_i = 0.001
blend_3 = 0.0
path_pose1 = [start_pos[0], start_pos[1], start_pos[2], start_pos[3], start_pos[4], start_pos[5], velocity, acceleration, blend_1]
path = []
path.append(path_pose1)
for i in range(data.shape[0]-1):
    path.append([positions_converted[i][0], positions_converted[i][1], positions_converted[i][2], positions_converted[i][3], positions_converted[i][4], positions_converted[i][5], velocity, acceleration, blend_i])

path_pose3 = [positions_converted[-1][0], positions_converted[-1][1], positions_converted[-1][2], positions_converted[-1][3], positions_converted[-1][4], positions_converted[-1][5], velocity, acceleration, blend_3]
path.append(path_pose3)

In [15]:
# Send a linear path with blending in between - (currently uses separate script)
rtde_c.moveL(path)
rtde_c.stopScript()